In [1]:
import torch
import torchaudio
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import torch
from torch.nn import Module, Parameter
from torch import FloatTensor
from scipy import signal
import numpy as np
from torchaudio import transforms
import matplotlib.pyplot as plt
import IPython.display as ipd
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
import os


In [2]:
# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("device=", device) 

device= cpu


In [3]:
train_data_dir = './data/train'
val_data_dir = './data/val'
input_name = 'ht1-input.wav'
target_name = 'ht1-target.wav'
train_sig, _ = torchaudio.load(os.path.join(train_data_dir, input_name))
val_sig, sr = torchaudio.load(os.path.join(val_data_dir, input_name))
train_target_sig, _ = torchaudio.load(os.path.join(train_data_dir, target_name))
val_target_sig, _ = torchaudio.load(os.path.join(val_data_dir, target_name))
print(sr)

44100


### Do we need transform here? maybe not?

In [79]:
# # transform : downsample here 
# new_sr = 8000 #10000
# transform = transforms.Resample(orig_freq=sr, new_freq=new_sr)
# train_input = transform(sig)
# train_target = transform(tar_sig)
# ipd.Audio(train_input.numpy(), rate=new_sr)

In [4]:
train_input = train_sig[:,:44100*60]
train_target = train_target_sig[:,:44100*60]

# For Reproducibility

In [5]:
torch.manual_seed(0)

## Initialize Dataloader

In [6]:
class DIIRDataSet(Dataset):
    def __init__(self, input, target, sequence_length):
        self.input = input
        self.target = target
        self._sequence_length = sequence_length
        self.input_sequence = self.wrap_to_sequences(self.input, self._sequence_length)
        self.target_sequence = self.wrap_to_sequences(self.target, self._sequence_length)
        self._len = self.input_sequence.shape[0]

    def __len__(self):
        return self._len

    def __getitem__(self, index):
        return {'input': self.input_sequence[index, :, :]
               ,'target': self.target_sequence[index, :, :]}

    def wrap_to_sequences(self, data, sequence_length):
        num_sequences = int(np.floor(data.shape[0] / sequence_length))
        print(num_sequences)
        truncated_data = data[0:(num_sequences * sequence_length)]
        wrapped_data = truncated_data.reshape((num_sequences, sequence_length, 1))
        wrapped_data = wrapped_data.permute(0,2,1)
        print(wrapped_data.shape)
        return np.float32(wrapped_data)


In [7]:
train_input.squeeze(0).shape

torch.Size([2646000])

In [8]:
batch_size = 512#1024
sequence_length = 512
train_dataset=DIIRDataSet(train_input.squeeze(0), train_target.squeeze(0), sequence_length)
loader = DataLoader(train_dataset, batch_size=batch_size, shuffle = False, pin_memory=True, drop_last=True) #? what does the shuffle really shuffles here?

5167
torch.Size([5167, 1, 512])
5167
torch.Size([5167, 1, 512])


In [9]:
len(loader)

10

# Declare Model

In [31]:
class FIRNN(Module):
    def __init__(self, n_input=1, n_output=1, kernel_size=80, n_channel=32):
        super(FIRNN, self).__init__()
        self.conv_kz = kernel_size
        self.input_len = 512
        self.conv1 = nn.Conv1d(n_input, n_channel, kernel_size=kernel_size, stride=1)
        self.nonlinear = nn.Tanh()
        self.bn1 = nn.BatchNorm1d(n_channel)

        self.fc1 = nn.Conv1d(n_channel, n_channel*2, kernel_size=1) 
        self.fc2 = nn.Conv1d(n_channel*2, n_channel, kernel_size=1)
        
        self.conv2 = nn.Conv1d(n_channel, n_output, kernel_size=kernel_size, stride=1)
        
        self.mlp_layer = nn.Sequential(
            self.fc1 ,
            nn.Tanh(),
            self.fc2,
        )


    def forward(self, x):
        #print(x.shape)
        bs = x.shape[0]
    
        # first get fir output
        x = F.pad(x, (self.conv_kz-1, 0)) #pad on the left side
        x = self.conv1(x) 
        #print(x.shape)
        x = self.nonlinear(self.bn1(x))
        #print(x.shape)

        #x = x.view(bs, -1)
        x = self.mlp_layer(x)
        #x = self.layers(x)
        #print(x.shape)
        
        x = F.pad(x, (self.conv_kz-1, 0))
        x = self.nonlinear(self.conv2(x))
        #print(x.shape)

        return x


In [32]:
model = FIRNN(kernel_size=80, n_channel=32)

## Define optimizer and criterion

In [33]:
import torch.nn as nn
from torch.optim import Adam

n_epochs = 100
lr = 1e-3

optimizer = Adam(model.parameters(), lr=lr, betas=(0.9, 0.999), eps=1e-08, weight_decay=0, amsgrad=False)

criterion = nn.MSELoss()

# Define train loop

In [34]:
def train(criterion, model, loader, optimizer):
    model.train()
    device = next(model.parameters()).device
    total_loss = 0
    
    for ind, batch in enumerate(loader):
        input_seq_batch = batch['input'].to(device)
        target_seq_batch = batch['target'].to(device)
        optimizer.zero_grad()
        predicted_output = model(input_seq_batch)
        loss = criterion(target_seq_batch, predicted_output)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    total_loss /= len(loader)
    return total_loss

## Train!

In [35]:
for epoch in range(n_epochs):
    loss = train(criterion, model, loader, optimizer)
    print("Epoch {} -- Loss {:3E}".format(epoch, loss))

Epoch 0 -- Loss 7.262408E-02
Epoch 1 -- Loss 6.325756E-02
Epoch 2 -- Loss 6.612604E-02
Epoch 3 -- Loss 6.803681E-02
Epoch 4 -- Loss 6.776053E-02
Epoch 5 -- Loss 6.346564E-02
Epoch 6 -- Loss 6.065446E-02
Epoch 7 -- Loss 6.015028E-02
Epoch 8 -- Loss 6.155975E-02
Epoch 9 -- Loss 6.044480E-02
Epoch 10 -- Loss 5.822162E-02
Epoch 11 -- Loss 5.794315E-02
Epoch 12 -- Loss 5.957253E-02
Epoch 13 -- Loss 6.114915E-02
Epoch 14 -- Loss 6.039316E-02
Epoch 15 -- Loss 5.939793E-02
Epoch 16 -- Loss 5.935268E-02
Epoch 17 -- Loss 5.711304E-02
Epoch 18 -- Loss 5.657114E-02
Epoch 19 -- Loss 5.626238E-02
Epoch 20 -- Loss 5.628797E-02
Epoch 21 -- Loss 5.826691E-02
Epoch 22 -- Loss 6.059146E-02
Epoch 23 -- Loss 5.993516E-02
Epoch 24 -- Loss 6.206088E-02
Epoch 25 -- Loss 6.466783E-02
Epoch 26 -- Loss 6.528736E-02
Epoch 27 -- Loss 6.446791E-02
Epoch 28 -- Loss 6.423380E-02
Epoch 29 -- Loss 6.200980E-02
Epoch 30 -- Loss 5.940140E-02
Epoch 31 -- Loss 5.847254E-02
Epoch 32 -- Loss 5.532814E-02
Epoch 33 -- Loss 5.6

# Evaluate

In [36]:
save_path = os.path.join('./models/model_WienerHammersteinNonlin_k80_bs512_ch32_44khz_ep{}.pth'.format(n_epochs-1))
torch.save(model.state_dict(), save_path)

In [37]:
val_batch_size = 128
sequence_length = 512
val_dataset=DIIRDataSet(train_input.squeeze(0), train_target.squeeze(0), sequence_length)
val_loader = DataLoader(val_dataset, batch_size=1, shuffle = False, pin_memory=True, drop_last=True)

5167
torch.Size([5167, 1, 512])
5167
torch.Size([5167, 1, 512])


In [38]:
def inspect_file(path):
    print("-" * 10)
    print("Source:", path)
    print("-" * 10)
    print(f" - File size: {os.path.getsize(path)} bytes")
    print(f" - {torchaudio.info(path)}")

In [39]:
def save_audio(batch):
    #1024,512,1
    out_batch = batch.detach().cpu()
    out_batch = out_batch.squeeze(-1).flatten()
    print(out_batch.shape)
    return out_batch

In [40]:
import soundfile as sf

out_path = './output/'
sample_rate = 44100
save_tensor = torch.zeros(5167,512)
with torch.no_grad():
    for i, val_batch in enumerate(val_loader):
        input_seq_batch = val_batch['input'].to(device)
        #target_seq_batch = val_batch['target'].to(device)
        predicted_output = model(input_seq_batch)
        output_tmp = predicted_output.squeeze().detach().cpu()
        #print(output_tmp.shape)
        save_tensor[i,:] = output_tmp
    
    print(save_tensor.shape)
    out_audio = save_audio(save_tensor)
    print(out_audio.shape)
    path = os.path.join(out_path, "target_WienerHammersteinNonlin_bs512_ch32_44khz.wav")
    print("Exporting {}".format(path))
    sf.write(path, out_audio, sample_rate,'PCM_24')
    #torchaudio.save(path, out_audio, sample_rate, encoding="PCM_S", bits_per_sample=16)
    inspect_file(path)
    

torch.Size([5167, 512])
torch.Size([2645504])
torch.Size([2645504])
Exporting ./output/target_WienerHammersteinNonlin_bs512_ch32_44khz.wav
----------
Source: ./output/target_WienerHammersteinNonlin_bs512_ch32_44khz.wav
----------
 - File size: 7936556 bytes
 - AudioMetaData(sample_rate=44100, num_frames=2645504, num_channels=1, bits_per_sample=24, encoding=PCM_S)


In [7]:
# #small test on padding
# t4d = torch.ones(3, 3, 4)
# print(t4d.shape)
# out = F.pad(t4d, (3,0)) #"constant", 0
# print(out.shape)

torch.Size([3, 3, 4])
torch.Size([3, 3, 7])


In [8]:
t4d[1,1,:]

tensor([1., 1., 1., 1.])

In [9]:
out[1,1,:]

tensor([0., 0., 0., 1., 1., 1., 1.])